<a href="https://colab.research.google.com/github/angel539/Python-Notebooks/blob/main/APIs_GoogleMaps_MongoDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Ángel Mora Segura** / *Científico de Datos* / [**Linkedin**](https://www.linkedin.com/in/angelmoras/)

Este notebook ha sido creado con un propósito educacional. Puedes compartir este notebook siempre que lo hagas bajo [estos términos](https://creativecommons.org/licenses/by-nc-sa/4.0/).

---

## 0. **Instalaciones**.

Docu: https://developers.google.com/places/web-service/client-library

**Instalando las dependencias:**

In [ ]:
import sys
print(sys.version)
# Importing mongo for 3.6.9 (default, Jul 17 2020, 12:50:27)

In [ ]:
# Instalando dependencias.
!{sys.executable} -m pip install -U googlemaps

In [ ]:
!{sys.executable} -m pip install folium

**Importando las librerias:**

In [ ]:
import googlemaps

In [ ]:
import pandas as pd

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from datetime import datetime

In [ ]:
import folium

## 1. **Operando con GoogleMaps API para Python**.

**Conectandose al cliente de GoogleMaps:**

In [ ]:
# Si quieres hacer esto en casa puedes consultar el siguiente enlace:
# https://github.com/googlemaps/google-maps-services-python
# Aquí tienes un ejemplo de conexión para cada una de las posibles APIs:
# https://github.com/googlemaps/google-maps-services-python/tree/master/tests

# Si quieres saber cómo generar tu propia clave de API puedes consultar el siguiente enlace:
# https://developers.google.com/maps/gmp-get-started
# Dentro de esa web pulsa el botón Get Started.
gmaps = googlemaps.Client(key='AQUÍ IRÍA LA KEY')

**Ejemplo #1: Usando `Geocoding API`**

Docu: https://developers.google.com/maps/documentation/geocoding/start

> Obteniendo la localización (latitud y longitud) de una dirección concreta.

In [ ]:
# Geocoding an address
geocode_result = gmaps.geocode('Plaza de España, 11, Madrid, España')
print(geocode_result)

> Buscando una dirección en un punto de localización concreto.

In [ ]:
# Look up an address with reverse geocoding
reverse_geocode_result = gmaps.reverse_geocode((40.4222658, -3.7162795))
print(reverse_geocode_result)

**Ejemplo #2: Usando `Directions API`**

Docu: https://developers.google.com/maps/documentation/directions/start

> Comprobando la ruta entre dos puntos.

In [ ]:
# Request directions via public transit
now = datetime.now()
directions_result = gmaps.directions("Plaza de España, 11, Madrid, España",
                                     "Plaza de Callao, Madrid, España",
                                     mode="transit",
                                     departure_time=now)
print(directions_result)

**Ejemplo #3: Usando `Places API`**

Docu: https://developers.google.com/places/web-service/overview

> Comprobando ciertos lugares de un tipo concreto.

In [ ]:
gmaps.find_place(
    input="Plaza de Callao, Madrid, España",
    input_type="textquery",
    language="es",
    fields=['place_id', 'name', 'geometry/location/lat', 'photos'])

In [ ]:
# Supported types:
# https://developers.google.com/places/supported_types
result_restaurantes = gmaps.places(query="Burger King",
                                    location = (40.4222658, -3.7162795),
                                    radius   = 30000,        # In meters
                                    language = "es",
                                    type     = "restaurant",
                                    open_now = False
                          )
# Complete result
for key, value in result_restaurantes.items():
      print(key, '->', value)

## 2. **Exportando el resultado a un `dataframe`, a un `mapa`, a una `base de datos` ...**

### 2.1 **De Maps a Pandas**.

In [ ]:
df_result_restaurantes = pd.DataFrame(columns = ['Place_Id', 'Nombre', 'Dirección', 'Icon', 'Latitude', 'Longitud', 'Tipo de lugar', 'Puntuación', 'Puntuaciones'])
df_result_restaurantes.head()

In [ ]:
for result in result_restaurantes.get("results"):
      print(result)

      new_row = {
          'Place_Id'      : result.get('place_id'),
          'Nombre'        : result.get('name'),
          'Dirección'     : result.get('formatted_address'),
          'Icon'          : result.get('icon'),
          'Latitude'      : result.get('geometry').get('location').get('lat'),
          'Longitud'      : result.get('geometry').get('location').get('lng'),
          'Tipo de lugar' : result.get('types'),
          'Puntuación'    : result.get('rating'),
          'Puntuaciones'  : result.get('user_ratings_total')
      }
      
      print(new_row)
      df_result_restaurantes = df_result_restaurantes.append(new_row, ignore_index=True)

In [ ]:
df_result_restaurantes.set_index('Place_Id')
df_result_restaurantes.head()

### 2.2 **De Pandas a Folium**.

In [ ]:
map = folium.Map(
    location = [40.4222658, -3.7162795],
    zoom_start = 6,
    tiles = 'stamentoner'
)
 
for index, row in df_result_restaurantes.iterrows():
    icon = folium.features.CustomIcon(row["Icon"], icon_size=(28, 30))
    
    folium.Marker(
              [row["Latitude"], row["Longitud"]],
              popup     = row["Nombre"] + "\n" + str(row["Puntuación"]) + " (" + str(row["Puntuaciones"]) + ") ", 
              tooltip   = row["Dirección"],
              icon      = icon
    ).add_to(map)

In [ ]:
map..save('mapa-google-maps.html')

### 2.3 **De Pandas a CSV**.

In [ ]:
df_result_restaurantes.to_csv("restaurantes.csv")

### 2.4 **De Pandas a MongoDB**.

> Conectandome a la base de datos de MongoDB:

In [ ]:
# Instalando dependencias.
!{sys.executable} -m pip install 'mongo' dnspython
!{sys.executable} -m pip install pymongo

In [ ]:
import pymongo
print ("version:", pymongo.version)

version: 3.11.0


In [ ]:
# Si quieres hacer esto en casa puedes consultar el siguiente enlace:
# https://docs.atlas.mongodb.com/tutorial/connect-to-your-cluster/

# Aquí, debes indicar la URL de conexión de tu cluster.

url = "URL del Cluster de MongoDB"
client = pymongo.MongoClient(url)
db = client.test

> Creando una nueva base de datos llamada `google_maps`:

In [ ]:
db = client.google_maps

> Insentando las filas en una colección `restaurantes`:

In [ ]:
for index, row in df_result_restaurantes.iterrows():
    nuevo_restaurante = {
          'Place_Id' :      row['Place_Id'],
          'Nombre' :        row['Nombre'],
          'Dirección' :     row['Dirección'],
          'Icon' :          row['Icon'],
          'Latitude' :      row['Latitude'],
          'Longitud' :      row['Longitud'],
          'Tipo de lugar' : row['Tipo de lugar'],
          'Puntuación':     row['Puntuación'],
          'Puntuaciones':   row['Puntuaciones']
    }

    result = db.restaurantes.insert_one(nuevo_restaurante)
    print("Insertado objeto: ", result.inserted_id)

In [ ]:
print(list(db.restaurantes.find()))

### 2.5 **De Maps a Pandas y MongoDB**.

**¿Y si ahora lo hacemos todo a la vez?** Es decir, ¿y si leemos de GoogleMaps, lo guardamos en un `dataframe` y lo guardamos en una base de datos de `MongoDB` a la vez?

In [ ]:
# Supported types:
# https://developers.google.com/places/supported_types
result_hospitales = gmaps.places(query="Hospital",
                        location = (40.4222658, -3.7162795),
                        radius   = 10000,                      # In meters
                        language = "es",
                        type     = "hospital"
                    )
# Complete result
for key, value in result_hospitales.items():
      print(key, '->', value)

In [ ]:
df_result_hospitales = pd.DataFrame(columns = ['Place_Id', 'Nombre', 'Dirección', 'Icon', 'Latitude', 'Longitud', 'Tipo de lugar', 'Puntuación', 'Puntuaciones'])
df_result_hospitales.head()

In [ ]:
for result in result_hospitales.get("results"):
      print(result)

      new_row = {
          'Place_Id'      : result.get('place_id'),
          'Nombre'        : result.get('name'),
          'Dirección'     : result.get('formatted_address'),
          'Icon'          : result.get('icon'),
          'Latitude'      : result.get('geometry').get('location').get('lat'),
          'Longitud'      : result.get('geometry').get('location').get('lng'),
          'Tipo de lugar' : result.get('types'),
          'Puntuación'    : result.get('rating'),
          'Puntuaciones'  : result.get('user_ratings_total')
      }
      
      df_result_hospitales = df_result_hospitales.append(new_row, ignore_index=True)

      result = db.hospitales.insert_one(new_row)
      print("Insertado objeto: ", result.inserted_id)

In [ ]:
print(list(db.hospitales.find()))

## 3. **`unwind()` + `Seaborn`**

[Docu con ejemplos](https://docs.mongodb.com/manual/reference/operator/aggregation/unwind/)

> Ejemplo de recuperando posiciones que están dentro de un array en MongoDB con `unwind()`:

In [ ]:
result_unwind1 = db.hospitales.aggregate([
                          { 
                              "$unwind" : "$Tipo de lugar"
                          }
])

for document in result_unwind1:
      print(document)

> Ejemplo de `unwind()` incluyendo el indice del array:

In [ ]:
result_unwind2 = db.hospitales.aggregate([
              { 
                  "$unwind" : {
                    "path": "$Tipo de lugar",
                    "includeArrayIndex": "arrayIndex"
                  }
              }
])

for document in result_unwind2:
      print(document)

> Desagrupando con `unwind()` para después agrupar por tipos y hacer la media de los ratings:

In [ ]:
result_unwind3 = db.hospitales.aggregate([
            # First Stage
            {
                "$unwind": { 
                    "path": "$Tipo de lugar", 
                    "preserveNullAndEmptyArrays": True
                }
            },
            # Second Stage
            {
                "$group": {
                    "_id": "$Tipo de lugar",
                    "averageRating": { 
                        "$avg": "$Puntuación"
                    }
                }
            },
            # Third Stage
            {
                "$sort": { 
                    "averageRating": -1
                }
            }
])

for document in result_unwind3:
  print(document)

> Hago lo de siempre: crear un dataframe vacio y rellenarlo.

In [ ]:
df_result_ratings = pd.DataFrame(columns = ['Id', 'Rating'])
df_result_ratings.head()

In [ ]:
result_unwind4 = db.restaurantes.aggregate([
            # First Stage
            {
                "$unwind": { 
                    "path": "$Tipo de lugar", 
                    "preserveNullAndEmptyArrays": True
                }
            },
            # Second Stage
            {
                "$group": {
                    "_id": "$Tipo de lugar",
                    "averageRating": { 
                        "$avg": "$Puntuación"
                    }
                }
            },
            # Third Stage
            {
                "$sort": { 
                    "averageRating": -1
                }
            }
])

for document in result_unwind4:
    new_row = {
        'Id'      : document.get("_id"),
        'Rating'  : document.get("averageRating")
    }
    df_result_ratings = df_result_ratings.append(new_row, ignore_index=True)

In [ ]:
df_result_ratings.head()

In [ ]:
fig, ax = plt.subplots(figsize=(15,6))
ax.set_ylim(df_result_ratings["Rating"].min()-0.05, df_result_ratings["Rating"].max()+0.05)
sns.barplot(x="Id", y="Rating", data=df_result_ratings, palette="rainbow")
plt.show()

## 4. Operando con `next_page_token`

Si observas nuestros resultados hasta ahora, sólo hemos conseguir recuperar los primeros 20 resultados en la búsqueda.

> Vamos a ver ahora como operar con `nextPageToken`.

In [ ]:
# Supported types:
# https://developers.google.com/places/supported_types
result_restaurantes_page1 = gmaps.places(query="Burger King",
                                        location = (40.4222658, -3.7162795),
                                        radius   = 30000,                      # In meters
                                        language = "es",
                                        type     = "restaurant",
                                        open_now = False
                    )
# Complete result
for key, value in result_restaurantes_page1.items():
      print(key, '->', value)

Vamos a repetir esta misma búsqueda con su `next_page_token`:

In [ ]:
result_restaurantes_page2 = gmaps.places(query="Burger King",
                        location   = (40.4222658, -3.7162795),
                        radius     = 30000,                      # In meters
                        language   = "es",
                        type       = "restaurant",
                        open_now   = False,
                        page_token = result_restaurantes_page1.get("next_page_token")
                    )
# Complete result
for key, value in result_restaurantes_page2.items():
      print(key, '->', value)

> ¿Y si lo hacemos como si fueramos programadores?

In [ ]:
result_restaurantes = gmaps.places(
                          query      = "Mc Donalds",
                          location   = (40.4222658, -3.7162795),
                          radius     = 30000,                      # In meters
                          language   = "es",
                          type       = "food",
                          open_now   = False
                      )

limit = 1
while (limit < 20) and ("next_page_token" in result_restaurantes):
    try:
      result_restaurantes = gmaps.places(
                                    query      = "Mc Donalds",
                                    page_token = result_restaurantes.get("next_page_token")
                            )
    except:
      limit = 20
    limit += 1

Vamos a probar ahora lo de MongoDB combinado con esto:

In [ ]:
def maps_to_mongo(result_maps):
  for result in result_maps.get("results"):
      new_row = {
          'Place_Id'      : result.get('place_id'),
          'Nombre'        : result.get('name'),
          'Dirección'     : result.get('formatted_address'),
          'Icon'          : result.get('icon'),
          'Latitude'      : result.get('geometry').get('location').get('lat'),
          'Longitud'      : result.get('geometry').get('location').get('lng'),
          'Tipo de lugar' : result.get('types'),
          'Puntuación'    : result.get('rating'),
          'Puntuaciones'  : result.get('user_ratings_total')
      }
      
      result = db.galerias.insert_one(new_row)
      print("Insertado objeto: ", result.inserted_id)

In [ ]:
result_restaurantes = gmaps.places(
                          query      = "Galería",
                          location   = (40.4222658, -3.7162795),
                          radius     = 50000,                      # In meters
                          language   = "es",
                          type       = "tourist_attraction",
                          open_now   = False
                      )
maps_to_mongo(result_restaurantes)

limit = 1
while (limit < 20) and ("next_page_token" in result_restaurantes):
    try:
      result_restaurantes = gmaps.places(
                                    query      = "Galería",
                                    page_token = result_restaurantes.get("next_page_token")
                            )
      maps_to_mongo(result_restaurantes)
    except:
      limit = 20
    limit += 1

## 5. `Places API` + `Directions API`

> Ahora vamos a juntar las consultas de la API de Places y las de la API de Direccions para crear una nueva tabla en MongoDB que nos posibilite guardar el recorrido entre dos puntos.

In [ ]:
def directions_to_mongo(origin, target, result_maps): 
  #for result in result_maps:
      # Aquí habría que hacer el mapping de los resultados obtenidos con Maps.
      print("From", origin, "To", target)
      print(result_maps) 
      #result_mongo = db.direcciones.insert_one(new_row)
      #print("Insertado objeto: ", result_mongo.inserted_id)

La respuesta de la API de direcciones tiene este formato:

https://developers.google.com/maps/documentation/directions/overview#DirectionsResponses

In [ ]:
now = datetime.now()

# Request directions via public transit
for hospital in db.hospitales.find().limit(10):
    for restaurante in db.restaurantes.find().limit(10):
        directions_result = gmaps.directions(
                                    (hospital.get("Latitude"), hospital.get("Longitud")),
                                    (restaurante.get("Latitude"), restaurante.get("Longitud")),
                                    mode="transit",
                                    departure_time = now)
        
        directions_to_mongo(hospital.get("Place_Id"), restaurante.get("Place_Id"), directions_result)

## 7. **Merging data in MongoDB**.

In [ ]:
# Para saber como operar con indices en MongoDB, puedes consultar el siguiente enlace:
# https://docs.mongodb.com/manual/indexes/
db.places.create_index([("place_id", pymongo.ASCENDING)], unique = True)

In [ ]:
result_q35 = db.galerias.aggregate([
            {
                "$project": {
                      "_id":            0,
                      "place_id":       "$Place_Id",
                      "name" :          "$Nombre",
                      "address" :       "$Dirección",  
                      "icon":           "$Icon",
                      "lat":            "$Latitude",
                      "lng":            "$Longitud",
                      "type":           "$Tipo de lugar",
                      "rating":         "$Puntuación",
                      "ratings_total":  "$Puntuaciones"
                  },
            },
            {
                "$merge": { 
                    "into":        "places",
                    "on"  :        "place_id",
                    "whenMatched": "keepExisting"
                }
            }
])

In [ ]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="can_explorer")
geo_spain = 'Spain'
location_spain = geolocator.geocode(geo_spain)

lat = location_spain.latitude
long = location_spain.longitude
print("Spain location: ", lat, long)

In [ ]:
map_places = folium.Map(
    location = [lat, long],
    zoom_start = 5,
    tiles = 'stamentoner'
)

for place in db.places.find():
    icon = folium.features.CustomIcon(place.get("icon"), icon_size=(15, 15))
    
    folium.Marker(
              [place.get("lat"), place.get("lng")],
              popup     = place.get("name") + "\n" + str(place.get("rating")) + " (" + str(place.get("ratings_total")) + ") ", 
              tooltip   = place.get("address"),
              icon      = icon
    ).add_to(map_places)

In [ ]:
map_places.save('places.html')